Multinomial Choice Models
In a typical linear regression model, we assume that the dependent variable is either continuous or binary. 
y_it=β_0+β_1 x_1it+β_2 x_2it+β_3 x_3it+λ_t+ϵ_it
An example of a continuous dependent variable might be the height of a tomato plant in inches. In this model, y_it is the height of the i^th tomato plant at time t, as a function of an intercept term, the amount of water (x_1), sunlight (x_2) and fertilizer (x_3) the plant receives, and a time trend variable. We interpret each coefficient β as the slope between its corresponding x and the value of y, holding all other independent variables constant.
An example of a binary dependent variable might be whether an individual attends college. In this model, y_it=1 corresponds to attending college for at least one semester, and y_it=0 corresponds to never attending college. We’d include an intercept term, and we’d also consider including covariates (independent variables) like family income (x_1), whether a parent attended college (x_2), high school GPA (x_3), and a time trend. We interpret each coefficient β as an increase or decrease to the probability of observing y = 1 for the dependent variable.
Both models are easy to estimate using linear regression models (such as OLS). But what happens when the choice faced by an individual doesn’t correspond to a binary outcome (for example, college/no college), but to multiple options from a categorical variable (for example, Apple iPhone / Android / Samsung Galaxy)? 
Multinomial choice models are used to provide mathematical structure to these problems. As with linear regression models, our goal is to estimate the parameters (coefficients) β that correspond to the covariates of interest x that we expect will influence the outcome. The output of the model is a set of probabilities that a given agent will choose one option compared to another. 
Multinomial choice models are used in practically every field of study. Brainstorm (or research) some applications of multinomial choice models in your chosen area (whether it’s your major, your anticipated career path, or just a field you find interesting). 


In [25]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# Load CSV
file_path = "transportation_ml.csv"  # Make sure this file is in your VS Code folder
df = pd.read_csv(file_path)

# Clean column names
df.columns = df.columns.str.strip().str.lower()
df = df.loc[:, ~df.columns.str.contains('^unnamed')]

# Convert from wide to long format (bike, bus, car)
long_df = pd.wide_to_long(df,
                          stubnames=['time', 'cost', 'risk'],
                          i='id',
                          j='mode',
                          sep='_',
                          suffix='.+').reset_index()

# Clean and merge choice column
df['choice'] = df['choice'].astype(str).str.strip().str.lower()
long_df['mode'] = long_df['mode'].astype(str).str.strip().str.lower()
long_df = long_df.merge(df[['id', 'choice']], on='id', how='left')

# Fix merge column if renamed
if 'choice_y' in long_df.columns:
    long_df.rename(columns={'choice_y': 'choice'}, inplace=True)

# Create "chosen" indicator
long_df['chosen'] = (long_df['mode'] == long_df['choice']).astype(int)

# Create dummy variables (bike = reference)
long_df = pd.get_dummies(long_df, columns=['mode'], drop_first=False)
if 'mode_bike' in long_df.columns:
    long_df.drop(columns=['mode_bike'], inplace=True)

# Define predictors (X) and response (y)
X = long_df[['time', 'cost', 'risk', 'mode_bus', 'mode_car']]
y = long_df['chosen']

# Add intercept and convert to float
X = sm.add_constant(X).astype(float)
y = y.astype(float)

# Drop any rows with missing values
data = pd.concat([X, y], axis=1).dropna()
X = data.drop(columns=[y.name])
y = data[y.name]

# Fit the multinomial logit model
model = sm.MNLogit(y, X)
result = model.fit(method='newton', maxiter=100, disp=True)

# Output summary
print("\n📊 MODEL SUMMARY:")
print(result.summary())

# Compute and print odds ratios
print("\n🔁 ODDS RATIOS (exp(coefficients)):")
print(np.exp(result.params))

# ---- BONUS: Predict probabilities for an average commuter ---- #
# Get mean time, cost, and risk
avg_time = long_df['time'].mean()
avg_cost = long_df['cost'].mean()
avg_risk = long_df['risk'].mean()

# Construct average commuter profiles (bike, bus, car)
avg_profiles = pd.DataFrame([
    [1, avg_time, avg_cost, avg_risk, 0, 0],  # bike
    [1, avg_time, avg_cost, avg_risk, 1, 0],  # bus
    [1, avg_time, avg_cost, avg_risk, 0, 1]   # car
], columns=['const', 'time', 'cost', 'risk', 'mode_bus', 'mode_car'],
   index=['Bike', 'Bus', 'Car'])

# Predict probabilities
predicted_probs = result.predict(avg_profiles)

# Print results
print("\n🚶 PREDICTED PROBABILITIES FOR AN AVERAGE COMMUTER:")
for mode, prob in zip(avg_profiles.index, predicted_probs):
    print(f"{mode}: {prob:.4f}")


Optimization terminated successfully.
         Current function value: 0.626882
         Iterations 5

📊 MODEL SUMMARY:
                          MNLogit Regression Results                          
Dep. Variable:                 chosen   No. Observations:                 3000
Model:                        MNLogit   Df Residuals:                     2994
Method:                           MLE   Df Model:                            5
Date:                Fri, 28 Mar 2025   Pseudo R-squ.:                 0.01513
Time:                        21:57:09   Log-Likelihood:                -1880.6
converged:                       True   LL-Null:                       -1909.5
Covariance Type:            nonrobust   LLR p-value:                 3.466e-11
  chosen=1       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.0212      0.265      3.848      0.000       0.501       1.541
time       

In [7]:
import numpy as np

# Simulated covariates (x_ij) for one individual and three alternatives (j = A, B, C)
x_A = np.array([1.0, 2.0])   # features for alternative A
x_B = np.array([1.5, 1.2])   # features for alternative B
x_C = np.array([0.8, 1.6])   # features for alternative C

# Coefficients β
beta = np.array([0.5, -0.3])

# Compute linear utility x_ij * beta
u_A = x_A @ beta
u_B = x_B @ beta
u_C = x_C @ beta

# Compute exp(x_ij * beta)
exp_A = np.exp(u_A)
exp_B = np.exp(u_B)
exp_C = np.exp(u_C)

# With all 3 alternatives
denominator_full = exp_A + exp_B + exp_C
pi_A_full = exp_A / denominator_full
pi_B_full = exp_B / denominator_full

# With only 2 alternatives: A and B (remove C)
denominator_AB = exp_A + exp_B
pi_A_ab = exp_A / denominator_AB
pi_B_ab = exp_B / denominator_AB

# Compare the ratios
odds_full = pi_A_full / pi_B_full
odds_ab = pi_A_ab / pi_B_ab

# Show that the ratio stays the same
print("π_A / π_B with all 3 choices   =", round(odds_full, 4))
print("π_A / π_B with only A and B    =", round(odds_ab, 4))
print("✅ Ratios equal (IIA holds)?    =", np.isclose(odds_full, odds_ab))

π_A / π_B with all 3 choices   = 0.6126
π_A / π_B with only A and B    = 0.6126
✅ Ratios equal (IIA holds)?    = True


## 	Can you prove algebraically that the relative probabilities of two choices do not depend on the presence or absence of other choices? (Hint: use the equation above, and find the ratio π_ij/π_ik for j≠k. 

In [2]:
import numpy as np
import pandas as pd

# Simulated β coefficients for 3 features
beta = np.array([0.5, -0.3, 0.2])

# Simulate covariates x_ij for 1 agent (i=1) and 3 alternatives (j=1,2,3)
x = {
    'Alt1': np.array([1.0, 2.0, 1.5]),
    'Alt2': np.array([1.2, 1.8, 1.0]),
    'Alt3': np.array([0.8, 2.2, 1.3])
}

# Compute utility and exp(xβ) for each alternative
utilities = {alt: x[alt] @ beta for alt in x}
exp_utilities = {alt: np.exp(utilities[alt]) for alt in x}

# Full choice set: Alt1, Alt2, Alt3
denominator_full = sum(exp_utilities.values())
prob_full = {alt: exp_utilities[alt] / denominator_full for alt in x}

# Subset choice set: Alt1, Alt2 only
denominator_subset = exp_utilities['Alt1'] + exp_utilities['Alt2']
prob_subset = {
    'Alt1': exp_utilities['Alt1'] / denominator_subset,
    'Alt2': exp_utilities['Alt2'] / denominator_subset
}

# Compute relative odds
odds_full = prob_full['Alt1'] / prob_full['Alt2']
odds_subset = prob_subset['Alt1'] / prob_subset['Alt2']

print("Odds (Alt1 vs Alt2) with 3 choices:", round(odds_full, 4))
print("Odds (Alt1 vs Alt2) with 2 choices:", round(odds_subset, 4))
print("Are odds equal?", np.isclose(odds_full, odds_subset))

Odds (Alt1 vs Alt2) with 3 choices: 0.9418
Odds (Alt1 vs Alt2) with 2 choices: 0.9418
Are odds equal? True


3. 

In [8]:
import pandas as pd

# Define voter groups and their rankings
voters = pd.DataFrame({
    'group': ['G1', 'G2', 'G3'],
    'percent': [0.25, 0.40, 0.35],
    '1st': ['A', 'B', 'C'],
    '2nd': ['B', 'C', 'A'],
    '3rd': ['C', 'A', 'B']
})

# First-past-the-post: count only first-choice votes
fptp_results = voters.groupby('1st')['percent'].sum().sort_values(ascending=False)
print("📊 First-Past-the-Post Results:")
print(fptp_results)
print(f"\n🏆 Winner (FPTP): {fptp_results.idxmax()}")

# Top-two runoff
# Step 1: Identify top two from first round (same as FPTP)
top_two = fptp_results.index[:2].tolist()
print(f"\n🧮 Top Two Candidates for Runoff: {top_two}")

# Step 2: Redistribute votes in runoff between top two
def runoff_vote(row, top_two):
    # Go through preferences in order
    for col in ['1st', '2nd', '3rd']:
        if row[col] in top_two:
            return row[col]
    return None

voters['runoff_vote'] = voters.apply(lambda row: runoff_vote(row, top_two), axis=1)

runoff_results = voters.groupby('runoff_vote')['percent'].sum()
print("\n📊 Runoff Results:")
print(runoff_results)
print(f"\n🏆 Winner (Runoff): {runoff_results.idxmax()}")


📊 First-Past-the-Post Results:
1st
B    0.40
C    0.35
A    0.25
Name: percent, dtype: float64

🏆 Winner (FPTP): B

🧮 Top Two Candidates for Runoff: ['B', 'C']

📊 Runoff Results:
runoff_vote
B    0.65
C    0.35
Name: percent, dtype: float64

🏆 Winner (Runoff): B


4. 	Set up and estimate a multinomial logit model to identify maximum-likelihood estimates of the parameter values β_time, β_cost, and β_risk. Identify which of these coefficients are statistically significant, if any, and interpret your findings. Note that, for technical reasons, you may have to transform the data by specifying one of the modes of transportation (“alternatives”) as the “pivot” or “reference” mode, and recoding the other variables as differences from the value corresponding to that “pivot” alternative. 

In [23]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# Step 1: Load and clean the dataset
file_path = "transportation_ml.csv"  # path to your local file in VS Code project
df = pd.read_csv(file_path)

# Clean column names and remove unnecessary ones
df.columns = df.columns.str.strip().str.lower()
df = df.loc[:, ~df.columns.str.contains("^unnamed")]

# Step 2: Transform wide to long format
long_df = pd.wide_to_long(df,
                          stubnames=['time', 'cost', 'risk'],
                          i='id',
                          j='mode',
                          sep='_',
                          suffix='.+').reset_index()

# Step 3: Prepare 'choice' and 'mode' columns
df['choice'] = df['choice'].astype(str).str.strip().str.lower()
long_df['mode'] = long_df['mode'].astype(str).str.strip().str.lower()

# Merge 'choice' column into long_df
long_df = long_df.merge(df[['id', 'choice']], on='id', how='left')

# Fix potential renaming during merge
if 'choice_y' in long_df.columns:
    long_df.rename(columns={'choice_y': 'choice'}, inplace=True)
elif 'choice_x' in long_df.columns:
    long_df.rename(columns={'choice_x': 'choice'}, inplace=True)

# Step 4: Create binary outcome for the chosen alternative
long_df['chosen'] = (long_df['mode'] == long_df['choice']).astype(int)

# Step 5: Create dummy variables for mode (use 'bike' as reference)
long_df = pd.get_dummies(long_df, columns=['mode'], drop_first=False)
if 'mode_bike' in long_df.columns:
    long_df = long_df.drop(columns=['mode_bike'])  # reference category

# Step 6: Define features (X) and target (y)
X = long_df[['time', 'cost', 'risk', 'mode_bus', 'mode_car']]
y = long_df['chosen']

# Add constant (intercept)
X = sm.add_constant(X)

# Step 7: Ensure numeric types and remove missing values
X = X.astype(float)
y = y.astype(float)
data = pd.concat([X, y], axis=1).dropna()
X = data.drop(columns=[y.name])
y = data[y.name]

# Step 8: Fit Multinomial Logit model using MLE
model = sm.MNLogit(y, X)
result = model.fit(method='newton', maxiter=100, disp=True)

# Step 9: Output results
print("\n📊 Model Summary:")
print(result.summary())

print("\n🔁 Odds Ratios (exp(coefficients)):")
print(np.exp(result.params))

Optimization terminated successfully.
         Current function value: 0.626882
         Iterations 5

📊 Model Summary:
                          MNLogit Regression Results                          
Dep. Variable:                 chosen   No. Observations:                 3000
Model:                        MNLogit   Df Residuals:                     2994
Method:                           MLE   Df Model:                            5
Date:                Fri, 28 Mar 2025   Pseudo R-squ.:                 0.01513
Time:                        21:31:41   Log-Likelihood:                -1880.6
converged:                       True   LL-Null:                       -1909.5
Covariance Type:            nonrobust   LLR p-value:                 3.466e-11
  chosen=1       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.0212      0.265      3.848      0.000       0.501       1.541
time       

## 5. estimated parameter values from the preceding question and estimate the probability that an “average” commuter (i.e., one whose values of “time”, “cost”, and “risk” are equal to the sample mean for each variable) would choose each mode of transportation. 

In [24]:
# Step 10: Predict probabilities for the average commuter

# Compute average values of the covariates across all rows
avg_time = long_df['time'].mean()
avg_cost = long_df['cost'].mean()
avg_risk = long_df['risk'].mean()

# Create input rows for each mode (bike, bus, car)
# Remember: we used dummy vars and dropped bike as reference
# So bike: mode_bus = 0, mode_car = 0
# bus:     mode_bus = 1, mode_car = 0
# car:     mode_bus = 0, mode_car = 1

avg_bike = [1, avg_time, avg_cost, avg_risk, 0, 0]
avg_bus  = [1, avg_time, avg_cost, avg_risk, 1, 0]
avg_car  = [1, avg_time, avg_cost, avg_risk, 0, 1]

# Stack into a DataFrame
avg_X = pd.DataFrame([avg_bike, avg_bus, avg_car],
                     columns=['const', 'time', 'cost', 'risk', 'mode_bus', 'mode_car'],
                     index=['Bike', 'Bus', 'Car'])

# Use fitted model to predict probabilities
probs = result.predict(avg_X)

# Display result
print("\n🚶‍♂️ Predicted Probabilities for Average Commuter:")
for mode, prob in zip(avg_X.index, probs):
    print(f"{mode}: {prob:.4f}")



🚶‍♂️ Predicted Probabilities for Average Commuter:
Bike: 0.0000
Bus: 1.0000
